# RSA implementation

Simple [PKCS1](https://www.ietf.org/rfc/rfc2437.txt) RSA implementation

In [200]:
import helpers

In [201]:
def modulus(p, q):
    return p*q

In [202]:
modulus(7, 13)

91

In [203]:
def rsa_public_exponent(p,q,n):
    lcm = helpers.least_common_multiple(p-1, q-1)
    for candidate in range (3, n):
        gcd, _, _ = helpers.extended_greatest_common_divisor(candidate, lcm)
        if gcd == 1:
            return candidate
        
print(rsa_public_exponent(7,13,91))

5


In [204]:
def rsa_private_exponent(p, q, e):
    return helpers.modular_inverse(e, (p-1)*(q-1))

print(rsa_private_exponent(7,13,5))

29


In [205]:
def find_one_large_prime():
    return 7

def find_another_large_prime():
    return 13

def rsa_generate_key_pair(p=None, q=None):
    p = p or find_one_large_prime()
    q = q or find_another_large_prime()
    
    n = modulus(p,q)
    
    public_key = n, rsa_public_exponent(p,q,n)
    
    e = public_key[1]
    
    private_key = n, rsa_private_exponent(p,q,e)
    
    return public_key, private_key

In [206]:
public_key, private_key = rsa_generate_key_pair()

print(public_key)
print(private_key)

(91, 5)
(91, 29)


In [207]:
def rsa_encrypt(message, public_key):
    n = public_key[0]
    e = public_key[1]
    return helpers.square_and_multiply(message, e, n)

In [208]:
rsa_encrypt(10, public_key)

82

In [209]:
def rsa_decrypt(cipher_text, private_key):
    n = private_key[0]
    d = private_key[1]
    return helpers.square_and_multiply(cipher_text, d, n)

In [210]:
rsa_decrypt(82, private_key)

10

In [211]:
def rsa_sign(message, private_key):
    n = private_key[0]
    d = private_key[1]
    return helpers.square_and_multiply(message, d, n)

In [212]:
rsa_sign(42, private_key)

35

In [213]:
def rsa_verify(signature, message, public_key):
    n = public_key[0]
    e = public_key[1]
    decrypted = rsa_decrypt(signature, public_key)
    if decrypted != message:
        raise Exception("Oh oh, invalid signature :sadpanda")    
    return True

In [214]:
rsa_verify(35, 42, public_key)

True

# Solving confidentiality, integrity and non-repudiation with RSA:

Alice wants to send "42" to Bob.

They want to make sure that:

1) No one besides Bob can understand the message (confidentiality)

2) The message Bob gets was not altered in the way (integrity)

3) Alice is really the person sending the message (non-repudiation)

### Adressing confidentiality with RSA

In [215]:
alice_public_key, alice_private_key = rsa_generate_key_pair(16468134996701, 15187421412743) # two random large primes
bob_public_key, bob_private_key = rsa_generate_key_pair(4993872162589, 3443848019779) # another two random large primes

print((alice_public_key, alice_private_key))
print((bob_public_key, bob_private_key))

def send_message_to_bob(message):
    return rsa_encrypt(message, bob_public_key)

((250108506076839141064360843, 3), (250108506076839141064360843, 166739004051204990338634267))
((17198136758161599975847831, 5), (17198136758161599975847831, 3439627351630632451133093))


In [216]:
send_message_from_alice_to_bob(42)

130691232

In [217]:
def bob_read_message(message):
    return rsa_decrypt(message, bob_private_key)

In [218]:
bob_read_message(130691232)

42

***No one besides Bob can understand the message as long as he is the only person in the world possessing the private key!!!***

# But how can Bob be sure that the sender is really Alice? How can he be sure that the message is not altered?

Answer: With encryption only, he can't!!! That's why we use signature!

In [219]:
def alice_send_message_to_bob(message):
    signature = rsa_sign(message, alice_private_key)
    encrypted_payload = rsa_encrypt(message, bob_public_key)
    return signature,encrypted_payload

In [220]:
# our man in the middle
charlie_public_key, charlie_private_key = rsa_generate_key_pair(14724824385277, 13121500152373)

print((charlie_public_key, charlie_private_key))

def charlie_send_message_to_bob(message):
    signature = rsa_sign(message, charlie_private_key)
    encrypted_payload = rsa_encrypt(message, charlie_public_key)
    return signature,encrypted_payload

((193211785415077821557812321, 5), (193211785415077821557812321, 77284714166019990093309869))


In [221]:
alice_send_message_to_bob(42)

(99863407955117330381311434, 130691232)

In [222]:
charlie_send_message_to_bob(42)

(60708453582239734568617826, 130691232)

In [223]:
charlie_send_message_to_bob(84)

(110930490035495277190753757, 4182119424)

In [224]:
def bob_read_message_from_alice(message):
    signature, encrypted_payload = message
    decrypted_message = rsa_decrypt(encrypted_payload, bob_private_key)
    rsa_verify(signature, decrypted_message, alice_public_key)
    return decrypted_message

In [225]:
bob_read_message_from_alice((99863407955117330381311434, 130691232))

42

In [226]:
bob_read_message_from_alice((60708453582239734568617826, 130691232))

Exception: Oh oh, invalid signature :sadpanda

In [1]:
bob_read_message_from_alice((99863407955117330381311434, 4182119424))

NameError: name 'bob_read_message_from_alice' is not defined

# OS2IP

***Signature solves both integrity AND non-repudiation problems!!!***

In [52]:
EB = [0x00,0x02,0xFB,0xEA,0xBC,0x21,0x38,0x4A,0xEE,0x51,0x00,0x88]

print(EB)

[0, 2, 251, 234, 188, 33, 56, 74, 238, 81, 0, 136]


In [48]:
def os2ip(EB):
    k = len(EB)
    sum = 0
    for i,val in enumerate(EB):        
        sum += pow(2,8*(k-i-1))*val
    return sum

In [49]:
os2ip(EB)

3607495720721035172053128

In [51]:
def i2osp(EB):

82